In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('titanic').getOrCreate()

In [3]:
data = spark.read.csv('titanic.csv',inferSchema=True,header=True)

In [4]:
data.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [5]:
data.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [6]:
my_cols = data.select(['Survived','Pclass','Sex','Age','SibSp','Parch','Fare','Embarked'])

In [8]:
my_final_data = my_cols.na.drop()

### Working with Categorical Columns
let us break this down into multiple steps to make it all clear.

In [9]:
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,OneHotEncoder,StringIndexer)

In [10]:
gender_indexer = StringIndexer(inputCol='Sex',outputCol='SexIndex')
gender_encoder = OneHotEncoder(inputCol='SexIndex',outputCol='SexVec')

In [20]:
embark_indexer = StringIndexer(inputCol='Embarked',outputCol='EmbarkIndex')
embark_encoder = OneHotEncoder(inputCol='EmbarkIndex',outputCol='EmbarkVec')

In [21]:
assembler =  VectorAssembler(inputCols=['Pclass','SexVec','Age','SibSp','Parch','Fare','EmbarkVec'],outputCol='features')

In [22]:
from pyspark.ml.classification import LogisticRegression

### Pipelines
Let us see an example of how to use pipelines.

In [23]:
from pyspark.ml import Pipeline

In [24]:
log_reg_titanic = LogisticRegression(featuresCol='features',labelCol='Survived')

In [25]:
pipeline = Pipeline(stages=[gender_indexer,embark_indexer,
                            gender_encoder, embark_encoder,
                            assembler, log_reg_titanic
])

In [26]:
train_titanic_data, test_titanic_data = my_final_data.randomSplit([0.7,0.3])

In [27]:
fit_model = pipeline.fit(train_titanic_data)

In [30]:
results = fit_model.transform(test_titanic_data)

In [31]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [32]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Survived')

In [33]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Survived')

In [34]:
results.select('Survived','prediction').show()

+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       1.0|
|       0|       0.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
+--------+----------+
only showing top 20 rows



In [35]:
AUC = my_eval.evaluate(results)

In [36]:
AUC

0.7303955586398334